# Data Wrangling: Join, Combine, and Reshape

# Init setup

In [144]:
import pandas as pd
import numpy as np

# 8.1 Hierarchical Indexing

- **Hierarchical indexing** is an important feature of `pandas` that enables you to have multiple index **`levels`** on an axis.
- It's provides a way for you to work with higher dimensional data in a lower dimensional form.

> ***Example - Hierarchical indexing***

In [145]:
data = pd.Series(
    np.random.rand(9),
    index=[
        list('aaabbcddd'),
        [1,2,3,1,3,3,4,2,1]
    ]
)
data

a  1    0.422015
   2    0.866415
   3    0.182381
b  1    0.021925
   3    0.404824
c  3    0.026982
d  4    0.042678
   2    0.626752
   1    0.083024
dtype: float64

In [146]:
# Select subsets of data
data['b':'c']

b  1    0.021925
   3    0.404824
c  3    0.026982
dtype: float64

In [147]:
data[['b','d']]

b  1    0.021925
   3    0.404824
d  4    0.042678
   2    0.626752
   1    0.083024
dtype: float64

In [148]:
data[:,2]

a    0.866415
d    0.626752
dtype: float64

- Hierarchical indexing plays an important role in reshaping data and group-based operations (e.g  forming a pivot table)

>  ***Example - Forming a pivot table from data with hierarchical indexing***

In [149]:
data = pd.Series(
    np.random.rand(9),
    index=[
        list('aaabbcddd'),
        [1,2,3,1,3,3,4,2,1]
    ]
)

data.unstack()

,1,2,3,4
a,0.019116,0.281391,0.503398,NaN
b,0.037969,NaN,0.485987,NaN
c,NaN,NaN,0.006482,NaN
d,0.084082,0.623581,NaN,0.494364


> ***Example - Convert pivot table to data with hierachical indexing***

In [150]:
data = pd.DataFrame(
    [
        list('abcd'),
        np.random.rand(4),
        list('efgh'),
        np.random.rand(4),
    ],
    columns=['Col_' + str(i + 1) for i in range(4)],
    index=['Row_' + str(i+1) for i in range(4)]
)
data

,Col_1,Col_2,Col_3,Col_4
Row_1,a,b,c,d
Row_2,0.202143,0.642016,0.0122519,0.351812
Row_3,e,f,g,h
Row_4,0.420916,0.30744,0.363264,0.925771


In [151]:
data.stack()

Row_1  Col_1            a
       Col_2            b
       Col_3            c
       Col_4            d
Row_2  Col_1     0.202143
       Col_2     0.642016
       Col_3    0.0122519
       Col_4     0.351812
Row_3  Col_1            e
       Col_2            f
       Col_3            g
       Col_4            h
Row_4  Col_1     0.420916
       Col_2      0.30744
       Col_3     0.363264
       Col_4     0.925771
dtype: object

### Reordering & Sorting Levels

> ***Example - Using `swaplevel()` & `sort_index()`***

In [152]:
data = pd.DataFrame(
    np.arange(12).reshape((4, 3)),
    index=[
        ['a', 'a', 'b', 'b'], 
        [1, 2, 1, 2]
    ],
    columns=[
        ['Ohio', 'Ohio', 'Colorado'],
        ['Green', 'Red', 'Green']
    ]
)
data.index.names = ['key1', 'key2']
data

Ohio     Colorado
          Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [153]:
data.swaplevel('key1','key2').sort_index(level=0)

Ohio     Colorado
          Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

### Summary Statistics by Level

- We can specify the **`level`** option in which we want to aggregate by on a particular axis.

> ***Example - Using `sum()` with `level` and `axis` options***

In [154]:
data

Ohio     Colorado
          Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [155]:
data.sum(level=1, axis=0)

Ohio     Colorado
     Green Red    Green
key2                   
1        6   8       10
2       12  14       16

In [156]:
data.sum(level=1, axis=1)

Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

### Indexing with a DataFrame's columns

> ***Example - Using `set_index()`***

In [157]:
data = pd.DataFrame({
    'a': range(7), 
    'b': range(7, 0, -1),
    'c': ['one', 'one', 'one', 'two', 'two','two', 'two'],
    'd': [0, 1, 2, 0, 1, 2, 3]
})
data

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [158]:
data.set_index(['c','d']) # we can add option drop=False to keep the column from being removed

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

## 8.2 Combining & Merging Datasets

### Database-Style DataFrame Joins

- **Merge** or **join** operations combine datasets by linking rows using one or more keys. These operations are central to relational databases.
- The `merge()` function in `pandas` is the main entry point for using these algorithms on your data.

> ***Example - Join two dataframes using `merge()`***

In [159]:
df1 = pd.DataFrame({
    'key': list('bbacaab'),
    'data1': range(7)
})
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [160]:
df2 = pd.DataFrame({
    'key': list('abd'),
    'data2': range(3)
})
df2

,key,data2
0,a,0
1,b,1
2,d,2


In [161]:
pd.merge(df1, df2) 
# If users don't specify a column to merge on,
# by default, merge() will use the overlap column in both DataFrame

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


- Different join types with `how` argument

| Option | Behavior |
|--------|----------|
| inner | Use only the key combinatins observed in both tables.<br>This is the default option|
| left | Use all key combinations found in the **left** table|
| right | Use all key combinations found in the **right** table|
| outer | Use all key combinations found in **both** table|

- `merge()` function arguments

| Argument | Description |
|----------|-------------|
| left | DataFrame to be merged on the left side|
| right | DataFrame to be merged on the right side|
| how | One of 'inner', 'outer', 'left', 'right'. Use 'inner' by default|
| on | Column names to join on. Must be found in both DataFrame objects.<br>If no specified and no other join keys given, <br>pandas will use the intersection of the column names in left and right as the join keys|
| left_on | Columns in left DataFrame to use as join keys|
| right_on | Columns in right DataFrame to use as join keys|
| left_index | Use row index in left as its join key |
| right_index | Use row index in right as its join key |
| sort | Sort merged data lexicographically by join keys; `True` by default.<br>(disable to get better performance on large datasets)|
| suffixes | Tuple of string values to append to column names in case of overlap;<br>defaults to ('_x','_y')|
| copy | If False, avoid copying data into resulting data structure in some exceptional cases.<br> By default always copies. |
| indicator | Adds a special column `_merge` that indicates the source of each row;<br>values will be `left_only`, `right_only`, or `both` based on the origin of the joined data in each row|

### Merging on Index

> ***Example - Join tables by using index***

In [162]:
data01 = pd.DataFrame({
    'key': list('abaabc'),
    'value': range(6)
})
data01

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [163]:
data02 = pd.DataFrame(
    {'group_val': [3.5, 7]},
    index=['a','b']
)
data02

,group_val
a,3.5
b,7.0


In [164]:
pd.merge(data01, data02, left_on='key', right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


> ***Example - Join tables with hierarchical index***

In [165]:
data01 = pd.DataFrame({
    'key1': ['Ohio', 'Ohio', 'Ohio','Nevada', 'Nevada'],
    'key2': [2000, 2001, 2002, 2001, 2002],
    'data': np.arange(5.)
})
data01

,key1,key2,data
0,Ohio,2000,0.0
1,Ohio,2001,1.0
2,Ohio,2002,2.0
3,Nevada,2001,3.0
4,Nevada,2002,4.0


In [166]:
data02 = pd.DataFrame(
    np.arange(12).reshape((6, 2)),
    index=[
        ['Nevada', 'Nevada', 'Ohio', 'Ohio','Ohio', 'Ohio'],
        [2001, 2000, 2000, 2000, 2001, 2002]
    ],
    columns=['event1', 'event2']
)
data02

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [167]:
pd.merge(data01, data02, left_on=['key1','key2'], right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4,5
0,Ohio,2000,0.0,6,7
1,Ohio,2001,1.0,8,9
2,Ohio,2002,2.0,10,11
3,Nevada,2001,3.0,0,1


### Concatenating Along an Axis

> ***Example - Using `concat()`***

In [168]:
s1 = pd.Series([0,1], index=['a','b'])
s2 = pd.Series([2,3,4], index=['c','d','e'])
s3 = pd.Series([5,6], index=['f','g'])

In [169]:
pd.concat([s1,s2,s3]) # By default, concat() works along axis 0 (row)

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [170]:
pd.concat([s1,s2,s3], axis=1, sort=False) # Concatenate along axis 1 (column)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [171]:
pd.concat([s1,s2], keys=['one','two']) 
# Concatenate and create hierarchical index on axis 0 (row)

one  a    0
     b    1
two  c    2
     d    3
     e    4
dtype: int64

In [172]:
pd.concat([s1,s2], keys=['one','two'], axis=1, sort=False) 
# On axis 1 (column), keys become column headers instead

,one,two
a,0.0,NaN
b,1.0,NaN
c,NaN,2.0
d,NaN,3.0
e,NaN,4.0


- `concat()` function arguments

| Argument | Description |
|----------|-------------|
| objs | List or dict of `pandas` objects to be concatenated|
| axis | axis to concatenate along; defaults to 0 |
| join | Either `inner` or `outer`; whether to intersection or union together indexes along the other axes; default to `outer`|
| join_axes | Specific indexes to use for the other `n-1` axes instead of performing union/intersection logic |
| keys | Values to associate with objects being concatenated, forming a hierarchical index along the concatenation axis; can either be a list or array of arbitrary values, an array of tuples, or a list of arrays|
| levels | Specific indexes to use as hierarchical index level or levels if keys passes|
| names | Names for created hierarchical levels if keys and/or levels passes|
| verify_integrity | Check new axis in concatenated object for duplicates and raise exception if so; by default (False) allows duplicates |
| ignore_index | Do not preserve indexes along concatenation axis, instead producing a new `range(total_leng)` index|

### Combining Data with Overlap

- There are data combination situations that can't be expressed as either a merge or concatenation operation.
- Consider using `where()` which performs the array-oriented equivalent of an if-else expression

> ***Example - Using `where()`***

In [173]:
s1 = pd.Series([np.nan, 1.4, np.nan, 3.4, 2.1, np.nan],
              index=list('abcdef'))
s1

a    NaN
b    1.4
c    NaN
d    3.4
e    2.1
f    NaN
dtype: float64

In [174]:
s2 = pd.Series(np.arange(len(s1), dtype=np.float64),
              index=list('abcdef'))
s2

a    0.0
b    1.0
c    2.0
d    3.0
e    4.0
f    5.0
dtype: float64

In [175]:
np.where(pd.isnull(s1), s2 , s1)

array([0. , 1.4, 2. , 3.4, 2.1, 5. ])

## 8.3 Reshaping & Pivoting

### Reshaping with Hierarchical Indexing

- Hierarchical indexing provides a consistent way to rearrange data in a DataFrame.
- There are 2 primary actions:
    1. `stack`: This **rotates** or pivots from the column in the data to the rows
    2. `unstack`: This pivot from the rows into the columns


> ***Example - Unstacking by level***

In [176]:
data = pd.DataFrame(
    np.arange(6).reshape((2,3)),
    index=pd.Index(
        ['Index 1', 'Index 2'],
        name='State'
    ),
    columns=pd.Index(
        ['one','two','three'],
        name='Number'
    )
)
data

Number,one,two,three
State,,,
Index 1,0,1,2
Index 2,3,4,5


In [177]:
data.stack()

State    Number
Index 1  one       0
         two       1
         three     2
Index 2  one       3
         two       4
         three     5
dtype: int32

In [178]:
data.stack().unstack(0)

State,Index 1,Index 2
Number,,
one,0,3
two,1,4
three,2,5


### Pivoting "Long" to "Wide" Format

- A common way to store multiple time series in Databases and CSV is in so-called `long` or `stacked` format.

> ***Example - Using `pivot()`***

In [179]:
data = pd.read_csv(r'https://raw.githubusercontent.com/wesm/pydata-book/2nd-edition/examples/macrodata.csv')
data.head()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [180]:
periods = pd.PeriodIndex(year=data.year, quarter=data.quarter, name='date')
periods

PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', name='date', length=203, freq='Q-DEC')

In [181]:
columns = pd.Index(['realgdp', 'infl', 'unemp'], name='item')
data = data.reindex(columns=columns)
data.head()

item,realgdp,infl,unemp
0,2710.349,0.00,5.8
1,2778.801,2.34,5.1
2,2775.488,2.74,5.3
3,2785.204,0.27,5.6
4,2847.699,2.31,5.2


In [182]:
data.index = periods.to_timestamp('D', 'end')
data.head()

item,realgdp,infl,unemp
date,,,
1959-03-31 23:59:59.999999999,2710.349,0.00,5.8
1959-06-30 23:59:59.999999999,2778.801,2.34,5.1
1959-09-30 23:59:59.999999999,2775.488,2.74,5.3
1959-12-31 23:59:59.999999999,2785.204,0.27,5.6
1960-03-31 23:59:59.999999999,2847.699,2.31,5.2


In [183]:
ldata = data.stack().reset_index().rename(columns={0: 'value'})
ldata.head()

,date,item,value
0,1959-03-31 23:59:59.999999999,realgdp,2710.349
1,1959-03-31 23:59:59.999999999,infl,0.000
2,1959-03-31 23:59:59.999999999,unemp,5.800
3,1959-06-30 23:59:59.999999999,realgdp,2778.801
4,1959-06-30 23:59:59.999999999,infl,2.340


- This is so-called `long` format for multiple time series, or other observational data with two or more key. Each row in the table represents a single observation.

In [184]:
ldata.pivot('date','item','value')

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2
...,...,...,...
2008-09-30 23:59:59.999999999,-3.16,13324.600,6.0
2008-12-31 23:59:59.999999999,-8.79,13141.920,6.9
2009-03-31 23:59:59.999999999,0.94,12925.410,8.1


### Pivoting "Wide" to "Long" Format

- An inverse operation to pivot for DataFrame is `pandas.melt`. Rather than transforming one column into many in a new Dataframe, it merges multiple columns into one, producing a DataFrame that is longer than the input.

> ***Example - Using `melt()`***

In [185]:
data = pd.DataFrame({
    'key': 'foo|bar|baz'.split('|'),
    'A': [1,2,3],
    'B': [4,5,6],
    'C': [7,8,9]
})
data

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [186]:
melted = pd.melt(data, ['key'])
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


In [187]:
reshaped = melted.pivot('key','variable','value') # Reshape back to the original layout
reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


In [188]:
pd.melt(data, id_vars=['key'], value_vars=['A','B'])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
